<a href="https://colab.research.google.com/github/Shashank-Holla/TSAI-END-Program/blob/main/08-%20HandsOn%202/RNN_French_To_German_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN based machine translation - French to German

RNN based model to predict German translations from French sentences.

Based on encoder-decoder architecture. Encoder learns from French sentence training data to generate context vector. Inputs to the encoder are hidden state from previous cell and current timestep's embedded input.

Decoder takes in the hidden state of previous decoder timestep and also the predicted word from the previous timestep or ground truth (teacher forcing). Hidden state of the decoder are fed to a linear layer to predict the actual word.

In [2]:
# Import necessary packages
import torch
import torch.nn as nn
import torch.optim as optim

# Extension of Flickr30K dataset. 
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import time
import math

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Download and load spacy models for DE and EN.

In [4]:
# Download and load spacy models for DE and FR languages. To be used for tokenization.
%%bash
python -m spacy download fr
python -m spacy download de

  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=21c5ea5515cb01193187128c860c6409b52712d82a19933a0c8753a9fa7760a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-d1bpu5xb/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=52b87f33611f3c2a19b8769a29c06bafad4b907ba3a15582108b97e96699477f
  Stored in directory: /tmp/pip-ephem-wheel-cache-fi1jnuzm/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now lo

In [5]:
# After linking spacy.load('de_core_news_sm') -> spacy.load('de')
spacy_de = spacy.load('de')
# After linking spacy.load('fr_core_news_sm') -> spacy.load('fr')
spacy_fr = spacy.load('fr')

### Prepare tokenizers

Takes in string and returns list of tokens. Will be passed to torchtext.

In the paper we are implementing, they find it beneficial to reverse the order of the input which they believe "introduces many short term dependencies in the data that make the optimization problem much easier".

In [6]:
# Tokenizer for French. Returns reversed tokens.
def tokenize_fr(text):
    """
    Tokenizes French text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_fr.tokenizer(text)][::-1]

def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

In [7]:
tokenize_fr("bonjour ma chérie!")

['!', 'chérie', 'ma', 'bonjour']

### Prepare FIELDS

Defines a datatype together with instructions for converting to Tensor. holds a Vocab object that defines the set of possible values
    for elements of the field and their corresponding numerical representations.

SRC is German and TRG is EN.

In [8]:
SRC = Field(tokenize = tokenize_fr, init_token = '<SOS>', eos_token = '<EOS>', lower = True)
TRG = Field(tokenize = tokenize_de, init_token = '<SOS>', eos_token = '<EOS>', lower = True)

In [9]:
SRC.__dict__

{'batch_first': False,
 'dtype': torch.int64,
 'eos_token': '<EOS>',
 'fix_length': None,
 'include_lengths': False,
 'init_token': '<SOS>',
 'is_target': False,
 'lower': True,
 'pad_first': False,
 'pad_token': '<pad>',
 'postprocessing': None,
 'preprocessing': None,
 'sequential': True,
 'stop_words': None,
 'tokenize': <function __main__.tokenize_fr>,
 'truncate_first': False,
 'unk_token': '<unk>',
 'use_vocab': True}

### Download Multi30K dataset and load train, test and val data.

Data stored under .data/multi30k/

Train and Validation data for french is not available in default pytorch download. Hence downloading the raw data and placing them in .data folder.

"https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.fr.gz"

"https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/val.fr.gz"

In [10]:
!wget "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.fr.gz"

--2020-12-17 17:35:43--  https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.fr.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 604242 (590K) [application/octet-stream]
Saving to: ‘train.fr.gz’

train.fr.gz         100%[===================>] 590.08K  --.-KB/s    in 0.05s   

2020-12-17 17:35:43 (12.2 MB/s) - ‘train.fr.gz’ saved [604242/604242]



In [11]:
!wget "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/val.fr.gz"

--2020-12-17 17:35:45--  https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/val.fr.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23001 (22K) [application/octet-stream]
Saving to: ‘val.fr.gz’

val.fr.gz           100%[===================>]  22.46K  --.-KB/s    in 0.002s  

2020-12-17 17:35:45 (13.1 MB/s) - ‘val.fr.gz’ saved [23001/23001]



In [15]:
mv train.fr.gz .data/multi30k/

In [13]:
mv val.fr.gz .data/multi30k/

In [16]:
!gzip -dk .data/multi30k/train.fr.gz

In [17]:
!gzip -dk .data/multi30k/val.fr.gz

In [18]:
ls .data/multi30k/

mmt_task1_test2016.tar.gz  train.de     training.tar.gz  val.fr.gz
test2016.de                train.en     val.de           validation.tar.gz
test2016.en                train.fr     val.en
test2016.fr                train.fr.gz  val.fr


In [19]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.fr', '.de'), fields = (SRC, TRG))

In [20]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [21]:
print(vars(train_data.examples[0]))

{'src': ['.', 'buissons', 'de', 'près', 'dehors', 'sont', 'blancs', 'hommes', 'jeunes', 'deux'], 'trg': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']}


In [22]:
sent = ' '.join([word for word in train_data.examples[0].__dict__['src'][::-1]])
sent

'deux jeunes hommes blancs sont dehors près de buissons .'

### Build vocabulary for SRC and TRG

Vocabulary will associate each unique tokens with an index. Only tokens that appear atleast 2 times are considered. Other such words are replaced by <UNK>. Vocab object built on only training examples.

In [23]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [24]:
vars(SRC)

{'batch_first': False,
 'dtype': torch.int64,
 'eos_token': '<EOS>',
 'fix_length': None,
 'include_lengths': False,
 'init_token': '<SOS>',
 'is_target': False,
 'lower': True,
 'pad_first': False,
 'pad_token': '<pad>',
 'postprocessing': None,
 'preprocessing': None,
 'sequential': True,
 'stop_words': None,
 'tokenize': <function __main__.tokenize_fr>,
 'truncate_first': False,
 'unk_token': '<unk>',
 'use_vocab': True,
 'vocab': <torchtext.vocab.Vocab at 0x7f51dae28a58>}

In [25]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 6462
Unique tokens in target (en) vocabulary: 7855


In [26]:
# SRC.vocab dict has the following keys - freqs(provides word and its frequencies), itos(mapping of integer to string), 
# stoi(mapping of string to its integer) and vectors()
print(SRC.vocab.__dict__.keys())
SRC.vocab.__dict__

dict_keys(['freqs', 'itos', 'stoi', 'vectors'])


{'freqs': Counter({'.': 27686,
          'buissons': 23,
          'de': 14013,
          'près': 800,
          'dehors': 600,
          'sont': 1941,
          'blancs': 207,
          'hommes': 1782,
          'jeunes': 656,
          'deux': 4068,
          'géant': 18,
          'poulies': 2,
          'système': 3,
          'un': 34942,
          'fonctionner': 7,
          'font': 364,
          'casque': 278,
          'en': 9878,
          'plusieurs': 448,
          'bois': 344,
          'maisonnette': 2,
          'une': 20624,
          'dans': 8059,
          'grimpe': 35,
          'fille': 1756,
          'petite': 618,
          'fenêtre': 133,
          'nettoyer': 22,
          'pour': 1298,
          'échelle': 56,
          'sur': 7957,
          'tient': 904,
          'se': 1983,
          'bleue': 501,
          'chemise': 662,
          'homme': 7888,
          'manger': 114,
          'à': 5326,
          'préparent': 57,
          'fourneaux': 5,
          '

### Create iterators for train, valid and test.

BucketIterator is used instead of Standard Iterator. BucketIterator defines an iterator that batches examples of similar lengths together.

Minimizes amount of padding needed while producing freshly shuffled batches for each new epoch.

batch.src - [ (128) , (128), .. 23 times]**bold text**

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
device

device(type='cuda')

In [29]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, device = device)

In [30]:
batch = next(iter(train_iterator))

In [31]:
# Batch contains src and trg. Can be accessed as batch.src and batch.trg
# batch.src groups the first element of all sequences in batch of 128, then groups second element of sequence in batch 128.
print(type(batch))
print(batch)
print("Length of a source sequence", len(batch.src))
print("Length of source at first index", len(batch.src[0]))
print("Values at source first index", batch.src[0])
print(batch.trg[1])

<class 'torchtext.data.batch.Batch'>

[torchtext.data.batch.Batch of size 128 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 26x128 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 23x128 (GPU 0)]
Length of a source sequence 26
Length of source at first index 128
Values at source first index tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([   8,   54,    5,    5,    5,    5,    8,    5,   13,    5,    5,    5,
           5,    5,    5, 2892,    5,    5,    5,   18,    5,    5,   18,    8,
        2213,    5,    5,    5,    5,    8,    8,   18,   18,    5,    5,   96,
           

### Build seq2seq model

building our model in three parts. The encoder, the decoder and a seq2seq model that encapsulates the encoder and decoder and will provide a way to interface with each.



#### Encoder

Input - X = {x1, x2, x3, .. xT}

Embedded input - E(X) = {e(x1), e(x2), ... e(xT)}

Hidden state - H = {h1, h2, h3,.... ,hT}

$$\begin{align*}
(h_t^1, c_t^1) &= \text{EncoderLSTM}^1(e(x_t), (h_{t-1}^1, c_{t-1}^1))\\
(h_t^2, c_t^2) &= \text{EncoderLSTM}^2(h_t^1, (h_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Note how only our hidden state from the first layer is passed as input to the second layer, and not the cell state.

So our encoder looks something like this: 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq2.png?raw=1)

We create this in code by making an `Encoder` module, which requires we inherit from `torch.nn.Module` and use the `super().__init__()` as some boilerplate code. The encoder takes the following arguments:
- `input_dim` is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
- `emb_dim` is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with `emb_dim` dimensions. 
- `hid_dim` is the dimensionality of the hidden and cell states.
- `n_layers` is the number of layers in the RNN.
- `dropout` is the amount of dropout to use. This is a regularization parameter to prevent overfitting. Check out [this](https://www.coursera.org/lecture/deep-neural-network/understanding-dropout-YaGbR) for more details about dropout.

We aren't going to discuss the embedding layer in detail during these tutorials. All we need to know is that there is a step before the words - technically, the indexes of the words - are passed into the RNN, where the words are transformed into vectors. To read more about word embeddings, check these articles: [1](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/), [2](http://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html), [3](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), [4](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/). 

The embedding layer is created using `nn.Embedding`, the LSTM with `nn.LSTM` and a dropout layer with `nn.Dropout`. Check the PyTorch [documentation](https://pytorch.org/docs/stable/nn.html) for more about these.

One thing to note is that the `dropout` argument to the LSTM is how much dropout to apply between the layers of a multi-layer RNN, i.e. between the hidden states output from layer $l$ and those same hidden states being used for the input of layer $l+1$.

In the `forward` method, we pass in the source sentence, $X$, which is converted into dense vectors using the `embedding` layer, and then dropout is applied. These embeddings are then passed into the RNN. As we pass a whole sequence to the RNN, it will automatically do the recurrent calculation of the hidden states over the whole sequence for us! Notice that we do not pass an initial hidden or cell state to the RNN. This is because, as noted in the [documentation](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM), that if no hidden/cell state is passed to the RNN, it will automatically create an initial hidden/cell state as a tensor of all zeros. 

The RNN returns: `outputs` (the top-layer hidden state for each time-step), `hidden` (the final hidden state for each layer, $h_T$, stacked on top of each other) and `cell` (the final cell state for each layer, $c_T$, stacked on top of each other).

As we only need the final hidden and cell states (to make our context vector), `forward` only returns `hidden` and `cell`. 

The sizes of each of the tensors is left as comments in the code. In this implementation `n_directions` will always be 1, however note that bidirectional RNNs (covered in tutorial 3) will have `n_directions` as 2.


In [32]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

#### Decoder

The Decoder class does a single step of decoding, i.e. it ouputs single token per time-step. 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq3.png?raw=1)

The `Decoder` class does a single step of decoding, i.e. it ouputs single token per time-step. The first layer will receive a hidden and cell state from the previous time-step, $(s_{t-1}^1, c_{t-1}^1)$, and feeds it through the LSTM with the current embedded token, $y_t$, to produce a new hidden and cell state, $(s_t^1, c_t^1)$. The subsequent layers will use the hidden state from the layer below, $s_t^{l-1}$, and the previous hidden and cell states from their layer, $(s_{t-1}^l, c_{t-1}^l)$. This provides equations very similar to those in the encoder.

$$\begin{align*}
(s_t^1, c_t^1) = \text{DecoderLSTM}^1(d(y_t), (s_{t-1}^1, c_{t-1}^1))\\
(s_t^2, c_t^2) = \text{DecoderLSTM}^2(s_t^1, (s_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Remember that the initial hidden and cell states to our decoder are our context vectors, which are the final hidden and cell states of our encoder from the same layer, i.e. $(s_0^l,c_0^l)=z^l=(h_T^l,c_T^l)$.

We then pass the hidden state from the top layer of the RNN, $s_t^L$, through a linear layer, $f$, to make a prediction of what the next token in the target (output) sequence should be, $\hat{y}_{t+1}$. 

$$\hat{y}_{t+1} = f(s_t^L)$$

The arguments and initialization are similar to the `Encoder` class, except we now have an `output_dim` which is the size of the vocabulary for the output/target. There is also the addition of the `Linear` layer, used to make the predictions from the top layer hidden state.

Within the `forward` method, we accept a batch of input tokens, previous hidden states and previous cell states. As we are only decoding one token at a time, the input tokens will always have a sequence length of 1. We `unsqueeze` the input tokens to add a sentence length dimension of 1. Then, similar to the encoder, we pass through an embedding layer and apply dropout. This batch of embedded tokens is then passed into the RNN with the previous hidden and cell states. This produces an `output` (hidden state from the top layer of the RNN), a new `hidden` state (one for each layer, stacked on top of each other) and a new `cell` state (also one per layer, stacked on top of each other). We then pass the `output` (after getting rid of the sentence length dimension) through the linear layer to receive our `prediction`. We then return the `prediction`, the new `hidden` state and the new `cell` state.

In [33]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        input = input.unsqueeze(0)
        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

#### Seq2Seq model

For the final part of the implemenetation, we'll implement the seq2seq model. This will handle: 
- receiving the input/source sentence
- using the encoder to produce the context vectors 
- using the decoder to produce the predicted output/target sentence

Our full model will look like this:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq4.png?raw=1)

The `Seq2Seq` model takes in an `Encoder`, `Decoder`, and a `device` (used to place tensors on the GPU, if it exists).

For this implementation, we have to ensure that the number of layers and the hidden (and cell) dimensions are equal in the `Encoder` and `Decoder`. This is not always the case, we do not necessarily need the same number of layers or the same hidden dimension sizes in a sequence-to-sequence model. However, if we did something like having a different number of layers then we would need to make decisions about how this is handled. For example, if our encoder has 2 layers and our decoder only has 1, how is this handled? Do we average the two context vectors output by the decoder? Do we pass both through a linear layer? Do we only use the context vector from the highest layer? Etc.

Our `forward` method takes the source sentence, target sentence and a teacher-forcing ratio. The teacher forcing ratio is used when training our model. When decoding, at each time-step we will predict what the next token in the target sequence will be from the previous tokens decoded, $\hat{y}_{t+1}=f(s_t^L)$. With probability equal to the teaching forcing ratio (`teacher_forcing_ratio`) we will use the actual ground-truth next token in the sequence as the input to the decoder during the next time-step. However, with probability `1 - teacher_forcing_ratio`, we will use the token that the model predicted as the next input to the model, even if it doesn't match the actual next token in the sequence.  

The first thing we do in the `forward` method is to create an `outputs` tensor that will store all of our predictions, $\hat{Y}$.

We then feed the input/source sentence, `src`, into the encoder and receive out final hidden and cell states.

The first input to the decoder is the start of sequence (`<sos>`) token. As our `trg` tensor already has the `<sos>` token appended (all the way back when we defined the `init_token` in our `TRG` field) we get our $y_1$ by slicing into it. We know how long our target sentences should be (`max_len`), so we loop that many times. The last token input into the decoder is the one **before** the `<eos>` token - the `<eos>` token is never input into the decoder. 

During each iteration of the loop, we:
- pass the input, previous hidden and previous cell states ($y_t, s_{t-1}, c_{t-1}$) into the decoder
- receive a prediction, next hidden state and next cell state ($\hat{y}_{t+1}, s_{t}, c_{t}$) from the decoder
- place our prediction, $\hat{y}_{t+1}$/`output` in our tensor of predictions, $\hat{Y}$/`outputs`
- decide if we are going to "teacher force" or not
    - if we do, the next `input` is the ground-truth next token in the sequence, $y_{t+1}$/`trg[t]`
    - if we don't, the next `input` is the predicted next token in the sequence, $\hat{y}_{t+1}$/`top1`, which we get by doing an `argmax` over the output tensor
    
Once we've made all of our predictions, we return our tensor full of predictions, $\hat{Y}$/`outputs`.

**Note**: our decoder loop starts at 1, not 0. This means the 0th element of our `outputs` tensor remains all zeros. So our `trg` and `outputs` look something like:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Later on when we calculate the loss, we cut off the first element of each tensor to get:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

In [34]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        # Dimensions to create output tensor to store results from decoder.
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)

        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        for t in range(1, trg_len):
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs



### Training the Seq2Seq model

First, we'll initialize our model. As mentioned before, the input and output dimensions are defined by the size of the vocabulary. The embedding dimesions and dropout for the encoder and decoder can be different, but the number of layers and the size of the hidden/cell states must be the same.

We then define the encoder, decoder and then our Seq2Seq model, which we place on the device.

In [35]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 3
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

initialize all weights from a uniform distribution between -0.08 and +0.08, i.e.  U(−0.08,0.08) .

We initialize weights in PyTorch by creating a function which we apply to our model. When using apply, the init_weights function will be called on every module and sub-module within our model. For each module we loop through all of the parameters and sample them from a uniform distribution with nn.init.uniform_.

In [36]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6462, 256)
    (rnn): LSTM(256, 512, num_layers=3, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=3, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [37]:
w = torch.zeros(3, 5)
print(w)
nn.init.uniform_(w)
print(w)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[0.0048, 0.0146, 0.0903, 0.0967, 0.1055],
        [0.0428, 0.0134, 0.0433, 0.3332, 0.5763],
        [0.7549, 0.5470, 0.9829, 0.0193, 0.7724]])


In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 19,253,679 trainable parameters


### Define optimizer and criterion

In [39]:
optimizer = optim.Adam(model.parameters())

we define our loss function. The CrossEntropyLoss function calculates both the log softmax as well as the negative log-likelihood of our predictions.

Our loss function calculates the average loss per token, however by passing the index of the <pad> token as the ignore_index argument we ignore the loss whenever the target token is a padding token.

In [40]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### Training loop

Decoder loop starts at 1, not 0. This means the 0th element of our `outputs` tensor remains all zeros. So our `trg` and `outputs` look something like:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Here, when we calculate the loss, we cut off the first element of each tensor to get:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

At each iteration:
- get the source and target sentences from the batch, $X$ and $Y$
- zero the gradients calculated from the last batch
- feed the source and target into the model to get the output, $\hat{Y}$
- as the loss function only works on 2d inputs with 1d targets we need to flatten each of them with `.view`
    - we slice off the first column of the output and target tensors as mentioned above
- calculate the gradients with `loss.backward()`
- clip the gradients to prevent them from exploding (a common issue in RNNs)
- update the parameters of our model by doing an optimizer step
- sum the loss value to a running total

Finally, we return the loss that is averaged over all batches.

In [41]:
def train(model, iterator, optimizer, creiterion, clip):
    model.train()

    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

### Testing loop

Evaluation loop is similar to training loop, however as we aren't updating any parameters we don't need to pass an optimizer or a clip value.

The iteration loop is similar (without the parameter updates), however we must ensure we turn teacher forcing off for evaluation. This will cause the model to only use it's own predictions to make further predictions within a sentence, which mirrors how it would be used in deployment.

In [42]:
def evaluate(model, iterator, criterion):
    model.eval()

    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) # Turning off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [43]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
    

In [44]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 54s
	Train Loss: 5.217 | Train PPL: 184.420
	 Val. Loss: 5.145 |  Val. PPL: 171.640
Epoch: 02 | Time: 0m 54s
	Train Loss: 4.777 | Train PPL: 118.759
	 Val. Loss: 5.182 |  Val. PPL: 178.044
Epoch: 03 | Time: 0m 54s
	Train Loss: 4.493 | Train PPL:  89.401
	 Val. Loss: 5.007 |  Val. PPL: 149.486
Epoch: 04 | Time: 0m 54s
	Train Loss: 4.220 | Train PPL:  68.031
	 Val. Loss: 4.889 |  Val. PPL: 132.835
Epoch: 05 | Time: 0m 54s
	Train Loss: 4.029 | Train PPL:  56.198
	 Val. Loss: 4.689 |  Val. PPL: 108.706
Epoch: 06 | Time: 0m 54s
	Train Loss: 3.891 | Train PPL:  48.935
	 Val. Loss: 4.609 |  Val. PPL: 100.429
Epoch: 07 | Time: 0m 54s
	Train Loss: 3.800 | Train PPL:  44.711
	 Val. Loss: 4.430 |  Val. PPL:  83.919
Epoch: 08 | Time: 0m 55s
	Train Loss: 3.660 | Train PPL:  38.879
	 Val. Loss: 4.365 |  Val. PPL:  78.688
Epoch: 09 | Time: 0m 55s
	Train Loss: 3.548 | Train PPL:  34.729
	 Val. Loss: 4.238 |  Val. PPL:  69.248
Epoch: 10 | Time: 0m 54s
	Train Loss: 3.434 | Train PPL

In [45]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.093 | Test PPL:  59.902 |


### Inferencing on sample sentences

In [47]:
sentence = "L'homme traverse la route."

In [51]:
tokenized = tokenize_fr(sentence)
tokenized

['.', 'route', 'la', 'traverse', 'homme', "L'"]

In [71]:
indexed = [SRC.vocab.__dict__['stoi'][t] for t in tokenized]
indexed

[5, 182, 16, 479, 12, 0]

In [89]:
length = len(indexed)
print(length)
 #src = [src len, batch size]
input_tensor = torch.LongTensor(indexed)
print(input_tensor.type())
print(input_tensor.shape)
input_tensor = input_tensor.unsqueeze(1).to(device).long()
print(input_tensor.shape)
print(input_tensor)
#trg = [trg len, batch size]
dummy_target = torch.zeros((length, 1)).to(device)
# dummy_target = torch.zeros(length).to(device)
print(dummy_target.shape)

6
torch.LongTensor
torch.Size([6])
torch.Size([6, 1])
tensor([[  5],
        [182],
        [ 16],
        [479],
        [ 12],
        [  0]], device='cuda:0')
torch.Size([6, 1])


In [86]:
model = model.eval()
prediction = model(input_tensor, dummy_target, 0)

RuntimeError: ignored